<a href="https://colab.research.google.com/github/ysn2496/Final_Project_huggingface/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers gradio torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 472.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130

In [ ]:
!pip install sentencepiece

In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


Using device: cuda


In [ ]:
import gradio as gr
from transformers import pipeline, MarianMTModel, MarianTokenizer
from diffusers import StableDiffusionPipeline
import torch

# Load the model and tokenizer for multilingual to English translation
model_name = 'Helsinki-NLP/opus-mt-mul-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

def translate_to_english(text):
    # Prepare input for the model
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generate the translation
    translated_tokens = model.generate(inputs['input_ids'], max_length=100)

    # Decode the translated tokens to get the output text
    translated_text = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return translated_text

# Step 2: Summarization Model using facebook/bart-large-cnn
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.cuda.is_available() else -1)

def summarize_text(translated_text):
    # Generate summary (adjust max_length and min_length as needed)
    summary = summarizer(translated_text, max_length=150, min_length=50, do_sample=False)[0]['summary_text']
    return summary



# Step 3: Text-to-Image Model (Stable Diffusion)
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16).to(device)

# Function to process input: translation -> summarization -> image generation
def process_text(tamil_text):
    try:
        # Progress update for UI (Can be handled separately in a more complex UI)
        progress = "Translating Tamil to English..."

        # Translation using Helsinki model
        translated_text = translate_to_english(tamil_text)

        progress = "Summarizing the translated text..."
        # Summarization of translated text using Pegasus
        summary = summarize_text(translated_text)

        progress = "Generating the image based on the summary..."
        # Image generation from the summary
        image = pipe(summary).images[0]

        return translated_text, summary, image
    except Exception as e:
        return "Error: " + str(e), "Error: " + str(e), None

# Gradio interface
with gr.Blocks() as app:
    gr.Markdown("# Tamil-to-English Translation, Summarization, and Image Generation")

    with gr.Row():
        tamil_input = gr.Textbox(label="Enter Tamil Text", placeholder="Enter text in Tamil here...")

    with gr.Row():
        translated_output = gr.Textbox(label="Translated Text (English)")
        summarized_output = gr.Textbox(label="Summarized Text")

    with gr.Row():
        generated_image = gr.Image(label="Generated Image")

    # Progress indicator
    progress_info = gr.Markdown("Awaiting input...")

    # Button to trigger the process
    generate_button = gr.Button("Generate")

    # Action on button click: updates UI with progress, performs tasks, and displays results
    generate_button.click(fn=process_text, inputs=[tamil_input], outputs=[translated_output, summarized_output, generated_image])

# Launch the app with a shareable link
app.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

...

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0c80995da886b03401.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import requests
import os

# Hugging Face API key (set in your Hugging Face account settings)
HF_API_KEY = os.getenv("HF_API_KEY")

# API URLs for the models
translation_model_url = "https://api-inference.huggingface.co/models/Helsinki-NLP/opus-mt-mul-en"
summarization_model_url = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"
stable_diffusion_model_url = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"

# Headers with authorization token
headers = {"Authorization": f"Bearer {HF_API_KEY}"}

# Function to call Hugging Face Inference API
def query_hf_api(model_url, payload):
    response = requests.post(model_url, headers=headers, json=payload)
    return response.json()

# Step 1: Translation using Helsinki-NLP/opus-mt-mul-en (Inference API)
def translate_to_english(text):
    payload = {"inputs": text}
    output = query_hf_api(translation_model_url, payload)
    return output[0]['translation_text'] if isinstance(output, list) and 'translation_text' in output[0] else str(output)

# Step 2: Summarization using facebook/bart-large-cnn (Inference API)
def summarize_text(translated_text):
    payload = {"inputs": translated_text, "parameters": {"max_length": 100, "min_length": 30}}
    output = query_hf_api(summarization_model_url, payload)
    return output[0]['summary_text'] if isinstance(output, list) and 'summary_text' in output[0] else str(output)

# Step 3: Text-to-Image generation using black-forest-labs/FLUX.1-dev (Inference API)
def generate_image_from_text(summary_text):
    payload = {"inputs": summary_text}
    response = requests.post(stable_diffusion_model_url, headers=headers, json=payload)
    if response.status_code == 200:
        # Save the image content from the response
        image_path = "generated_image.png"
        with open(image_path, "wb") as f:
            f.write(response.content)
        return image_path
    return None

# Main process function
def process_text(tamil_text):
    try:
        # Translation
        translated_text = translate_to_english(tamil_text)

        # Summarization
        summary = summarize_text(translated_text)

        # Image Generation
        image = generate_image_from_text(summary)

        return translated_text, summary, image
    except Exception as e:
        return f"Error: {str(e)}", f"Error: {str(e)}", None

# Gradio Interface
with gr.Blocks() as app:
    gr.Markdown("# Tamil-to-English Translation, Summarization, and Image Generation")

    with gr.Row():
        tamil_input = gr.Textbox(label="Enter Tamil Text", placeholder="Enter text in Tamil here...")

    with gr.Row():
        translated_output = gr.Textbox(label="Translated Text (English)")
        summarized_output = gr.Textbox(label="Summarized Text")

    with gr.Row():
        generated_image = gr.Image(label="Generated Image")

    # Button to trigger the process
    generate_button = gr.Button("Generate")

    # Action on button click
    generate_button.click(fn=process_text, inputs=[tamil_input], outputs=[translated_output, summarized_output, generated_image])

# Launch the app
app.launch()
